In [1]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore", "is_categorical_dtype")
warnings.filterwarnings("ignore", "use_inf_as_na")

In [2]:
prefix = r"./data/"
patient_df = pd.read_csv(prefix + "patient.csv", sep=",")
disease_df = pd.read_csv(prefix + "disease.csv", sep=",")
medicine_df = pd.read_csv(prefix + "medicine.csv", sep=",")
patient_disease_df = pd.read_csv(prefix + "patient-disease.csv", sep=",")
patient_medicine_df = pd.read_csv(prefix + "patient-medicine.csv", sep=",")


### Step 1 离散数据连续化
- 离散属性显然不能直接用于训练，因此需要将其通过one-hot编码或其他处理方式转换成连续数据。
- 从结果看来，存在一些非离散数据，主要是以`>`开头的一些数字和一些抗体的检测结果。对于前者，直接转换为数字。对于后者，进行one-hot编码。

In [3]:
disc_set = {col: set() for col in patient_df.columns}


def check_discrete(cell, col_name):
    try:
        float(cell)
    except ValueError:
        disc_set[col_name].add(cell)


for col in patient_df.columns:
    patient_df[col].apply(lambda cell: check_discrete(cell, col))

pd.DataFrame.from_dict(
    {col: list(disc_set[col]) for col in disc_set if len(disc_set[col]) != 0},
    orient="index",
).T

,胰岛素-空腹 数值,胰岛素-餐后60 数值,胰岛素-餐后120 数值,促甲状腺激素 数值,游离甲状腺素 数值,甲状腺球蛋白抗体 数值,抗甲状腺过氧化酶抗体 数值,促甲状腺素受体抗体 数值,孕酮 数值,雌二醇 数值,...,血清总I型胶原氨基末端肽测定 数值,癌胚抗原 数值,糖类抗原125 数值,糖类抗原19-9 数值,糖类抗原72-4 数值,糖类抗原242 数值,铁蛋白 数值,抗谷氨酸脱羧酶抗体(GAD-Ab) 数值,胰岛细胞抗体 数值,抗胰岛素抗体(IAA) 数值
0,>1000.0,>1000.0,>1000.0,>100.0,>100.0,>4000.0,>600.0,>40.0,>127,>7342,...,>1200.0,>100.00,>600.0,>1000.0,>300.0,>200,>1650.0,>2000,阴性(-),阴性(-)
1,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,阴性（－）,阴性(－)
2,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,阴性(－),阳性(+)
3,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,阳性(+),弱阳性(±)
4,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,弱阳性(±),阳性(＋)
5,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,阳性(＋),可疑
6,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,可疑,None


In [4]:
for col in patient_df.columns:
    for idx, cell in enumerate(patient_df[col]):
        try:
            patient_df.at[idx, col] = float(cell)  
        except ValueError:
            if '>' in cell:
                patient_df.at[idx, col] = float(cell[1:]) 
            elif '阴性' in cell:
                patient_df.at[idx, col] = 1.0
            elif '弱阳性' in cell:
                patient_df.at[idx, col] = 2.0
            elif '阳性' in cell:
                patient_df.at[idx, col] = 3.0
            elif '可疑' in cell:
                patient_df.at[idx, col] = 0
            else:
                raise
patient_df.head()

,ID,性别 数值,入院体重指数 数值,入院收缩压,院舒张压,入院腰围 数值,导出年龄,发病年龄,胰岛素-空腹 数值,胰岛素-餐后30 数值,...,感染,糖尿病酮症,糖尿病视网膜病变,糖尿病肾病,糖尿病周围神经病变,下肢动脉病变,颈动脉病变,脑血管病,冠心病,高血压病
0,1,1,24.91,137.0,71.0,83.0,73,69,-1.0,-1.0,...,0,0,0,0,0,0,0,0,0,1
1,2,1,24.00,147.0,84.0,88.0,69,59,-1.0,-1.0,...,0,0,0,1,1,1,0,0,0,1
2,3,2,30.50,171.0,81.0,104.0,60,49,-1.0,-1.0,...,0,0,1,1,1,0,1,1,0,1
3,4,1,29.30,108.0,50.0,NaN,81,61,-1.0,-1.0,...,1,0,0,0,0,0,0,1,1,1
4,5,1,25.30,139.0,101.0,100.0,42,40,-1.0,-1.0,...,0,0,0,0,0,0,0,0,0,1


### Step 2 空值检测
- 由于从知识图谱导出数据时以-1来代替了空值，所以先将所有的-1替换成`np.nan`，再统计数量。
- 从结果看来，存在大量的空值，因此为确保之后训练效果不受空值影响，需要填充缺失值。策略有常值填充、基于统计学的方法（正态分布+随机采样）

In [5]:
patient_df.replace({"-1": np.nan, -1: np.nan}, inplace=True)
print(f"Total : {patient_df.shape[0]} rows x {patient_df.shape[1]} columns")
missing_count = patient_df.isna().sum()

print(
    f"{missing_count[missing_count != 0].shape[0]}/{patient_df.shape[1]} columns have missnig values"
)
pd.DataFrame(missing_count[missing_count != 0] / patient_df.shape[0]).T.style.format(
    "{:.2%}"
).hide(axis="index")

patient_df.to_csv(prefix + "patient-cleaned.csv", index=False)

Total : 4388 rows x 84 columns
68/84 columns have missnig values


能够使用正态分布填充的先决条件是数据必须满足正态分布，因此检验数据是否符合正太分布至关重要。

我们先去除异常值（四分位数或Z-score），再通过Shapiro-Wilk测试和Kolmogorov-Smirnov测试来检查数据的正态性。

从结果看来，大部分的数据都是不符合正态分布的。如果使用正太分布可能会改变原始数据分布，导致模型学到错误的潜在分布。所以对于不满足正态分布的列，我们使用随机采样的方式填充。

In [6]:
from scipy.stats import kstest, shapiro

test_result = pd.DataFrame(index=missing_count[missing_count != 0].index).T
for col in test_result.columns:
    Q1 = patient_df[col].quantile(0.25)
    Q3 = patient_df[col].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    data_clean_iqr = patient_df[
        (patient_df[col] >= lower_bound) & (patient_df[col] <= upper_bound)
    ]
    
    test_result[col] = [(
        shapiro(data_clean_iqr[col].dropna())
        if data_clean_iqr.shape[0] <= 50
        else kstest(
            data_clean_iqr[col].dropna(),
            "norm",
            args=(
                data_clean_iqr[col].mean(),
                data_clean_iqr[col].std() + 1e-8, # prevent dividing zero
            ),
        )
    )[1] > 0.05] # only when p-value > 0.05 can we consider that the original data conforms to the normal distribution

normal_distribution_cols = test_result.columns[test_result.any()].to_list()
print(normal_distribution_cols)
print(f"{len(normal_distribution_cols)}/{test_result.shape[1]} columns conform to the normal distribution")

['入院体重指数 数值 ', '胆固醇 数值', '游离三碘甲状腺原氨酸 数值', '游离甲状腺素 数值', '葡萄糖(餐后1h) 数值', '葡萄糖(餐后2h) 数值']
6/68 columns conform to the normal distribution


### Step 3 构造PyG所需的Data类型
本任务显然是异构图，结点类型为病人、药品和疾病，边类型是suffer-from和take-medicine。值得一提的是，异构图神经网络的效果有可能并不如简单的同构图上的GAT或GCN（KDD'21 Are we really making much progress? Revisiting, benchmarking,
and refining heterogeneous graph neural networks）。所以我同时准备了同构图`Data`和异构图`Data`以使用不同的网络进行测试。

#### 同构图
这一步包含以下步骤：
1. 把疾病、药品和病人视作相同结点，把患病、带药视作相同类型的边。
2. 根据不同策略填充缺失值。
3. 形成PyG所需的`Data`。

In [8]:
from src.tools import homo_data

homodata = homo_data(prefix, fill_mode="default")
homodata

c:\Users\Geoffrey Jiang\anaconda3\envs\ml\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Data(
  x=[9017, 83],
  edge_index=[2, 46005],
  y=[9017],
  node_labels={
    patient=0,
    disease=1,
    medicine=2,
  }
)

#### 异构图
这一步包含以下步骤：
1. 把疾病、药品和病人视作不同结点，把患病、带药视作不同类型的边。
2. 为疾病使用SetenceTransformers库中的语言模型编码，为药品生成全 0 的 $n \times n$ tensor作为特征，其中 $n$ 为药品种数。
3. 根据不同策略填充缺失值。
4. 形成PyG所需的`HeteroData`。

In [ ]:
from src.tools import hetero_data

heterodata = hetero_data(prefix, fill_mode="default")
heterodata

HeteroData(
  patient={ x=[4388, 83] },
  medicine={ x=[280, 280] },
  disease={ x=[4349, 384] },
  (patient, suffer, disease)={ edge_index=[2, 35673] },
  (patient, take, medicine)={ edge_index=[2, 10332] }
)

In [ ]:
homodata.validate(), heterodata.validate()

(True, True)